In [20]:
import enum
import torch
import esm
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel


#Input Type(s): Texts (Protein Sequences and Substrates)

#Input Parameters: 1D

#Other Properties Related to Input: Protein sequence represented
#as a string of canonical amino acids, of maximum length 1022.
#Longer sequences are automatically truncated to this length.

# For substrate maximum sequence length is 512


# Define sequences and substrates
sequence_1 = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'  #@param {type:"string"}
substrate_1 = 'C(C1C(C(C(C(O1)O)O)O)O)O'  #@param {type:"string"}

sequence_2 = ''  #@param {type:"string"}
substrate_2 = ''  #@param {type:"string"}

sequence_3 = ''  #@param {type:"string"}
substrate_3 = ''  #@param {type:"string"}

sequence_4 = ''  #@param {type:"string"}
substrate_4 = ''  #@param {type:"string"}

sequence_5 = ''  #@param {type:"string"}
substrate_5 = ''  #@param {type:"string"}

sequence_6 = ''  #@param {type:"string"}
substrate_6 = ''  #@param {type:"string"}

sequence_7 = ''  #@param {type:"string"}
substrate_7 = ''  #@param {type:"string"}

sequence_8 = ''  #@param {type:"string"}
substrate_8 = ''  #@param {type:"string"}

sequence_9 = ''  #@param {type:"string"}
substrate_9 = ''  #@param {type:"string"}

sequence_10 = ''  #@param {type:"string"}
substrate_10 = ''  #@param {type:"string"}

sequence_11 = ''  #@param {type:"string"}
substrate_11 = ''  #@param {type:"string"}

sequence_12 = ''  #@param {type:"string"}
substrate_12 = ''  #@param {type:"string"}

sequence_13 = ''  #@param {type:"string"}
substrate_13 = ''  #@param {type:"string"}

sequence_14 = ''  #@param {type:"string"}
substrate_14 = ''  #@param {type:"string"}

sequence_15 = ''  #@param {type:"string"}
substrate_15 = ''  #@param {type:"string"}

sequence_16 = ''  #@param {type:"string"}
substrate_16 = ''  #@param {type:"string"}

sequence_17 = ''  #@param {type:"string"}
substrate_17 = ''  #@param {type:"string"}

sequence_18 = ''  #@param {type:"string"}
substrate_18 = ''  #@param {type:"string"}

sequence_19 = ''  #@param {type:"string"}
substrate_19 = ''  #@param {type:"string"}

sequence_20 = ''  #@param {type:"string"}
substrate_20 = ''  #@param {type:"string"}
# input_sequences = (
#     sequence_1, sequence_2, sequence_3, sequence_4, sequence_5,
#     sequence_6, sequence_7, sequence_8, sequence_9, sequence_10,
#     sequence_11, sequence_12, sequence_13, sequence_14, sequence_15,
#     sequence_16, sequence_17, sequence_18, sequence_19, sequence_20)



In [21]:
!pip install fair-esm

# Load ESM-2 model
esm_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
esm_model.eval()  # Disable dropout for deterministic results

# Load ChemBERT model
chembert_model = AutoModel.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
chembert_tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
chembert_model.eval()



# Prepare input data with both sequences and substrates
input_data = [
    ("sequence_1", sequence_1, substrate_1), ("sequence_2", sequence_2, substrate_2),
    ("sequence_3", sequence_3, substrate_3), ("sequence_4", sequence_4, substrate_4),
    ("sequence_5", sequence_5, substrate_5), ("sequence_6", sequence_6, substrate_6),
    ("sequence_7", sequence_7, substrate_7), ("sequence_8", sequence_8, substrate_8),
    ("sequence_9", sequence_9, substrate_9), ("sequence_10", sequence_10, substrate_10),
    ("sequence_11", sequence_11, substrate_11), ("sequence_12", sequence_12, substrate_12),
    ("sequence_13", sequence_13, substrate_13), ("sequence_14", sequence_14, substrate_14),
    ("sequence_15", sequence_15, substrate_15), ("sequence_16", sequence_16, substrate_16),
    ("sequence_17", sequence_17, substrate_17), ("sequence_18", sequence_18, substrate_18),
    ("sequence_19", sequence_19, substrate_19), ("sequence_20", sequence_20, substrate_20)
]

# Filter out entries where both sequence and substrate are longer than 1
filtered_data = [(name, seq, subs) for name, seq, subs in input_data if len(seq) > 1 and len(subs) > 1]

# Prepare data for ESM-2: Use only the sequence part for ESM-2 processing
esm_data = [(name, seq) for name, seq, subs in filtered_data]

# Convert batch using ESM's batch converter
batch_labels, batch_strs, batch_tokens = batch_converter(esm_data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# Extract ESM-2 representations
esm_representations = []
with torch.no_grad():
    esm_results = esm_model(batch_tokens, repr_layers=[33], return_contacts=True)
    esm_token_representations = esm_results["representations"][33]

# Generate per-sequence ESM-2 representations via averaging (1280-dimensional vectors)
for i, tokens_len in enumerate(batch_lens):
    esm_seq_repr = esm_token_representations[i, 1:tokens_len - 1].mean(0).cpu().numpy()
    esm_representations.append(esm_seq_repr)

# Convert ESM-2 representations to a NumPy array and normalize based on original train set statistics
esm_representations = np.array(esm_representations)
esm_mean = -0.0006011285004206002
esm_std = 0.18902993202209473
esm_representations = (esm_representations - esm_mean) / esm_std

# Prepare data for ChemBERT: Use only the substrate part for ChemBERT processing
chembert_representations = []
for _, _, subs in filtered_data:
    chembert_inputs = chembert_tokenizer(subs, return_tensors="pt")
    with torch.no_grad():
        chembert_outputs = chembert_model(**chembert_inputs)
    chembert_seq_repr = chembert_outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    chembert_representations.append(chembert_seq_repr)

# Convert ChemBERT representations to a NumPy array and normalize based on original train set statistics
chembert_representations = np.array(chembert_representations)
chembert_mean = -0.00015002528380136937
chembert_std = 0.6113553047180176
chembert_representations = (chembert_representations - chembert_mean) / chembert_std

# Concatenate normalized ESM-2 and ChemBERT representations
sequence_representations = np.concatenate([esm_representations, chembert_representations], axis=1)


# Load the XGBoost model from a pickle file
with open('xgb_y1_model.pkl', 'rb') as file:
    xgb_y1 = pickle.load(file)
predictions_y1 = xgb_y1.predict(sequence_representations)


with open('xgb_y2_model.pkl', 'rb') as file:
    xgb_y2 = pickle.load(file)
predictions_y2 = xgb_y2.predict(sequence_representations)


# Define low and high value ranges for each class (0 to 8)
class_ranges_y1 = {
    0: {"low": 0.0, "high": 3.32e-8},    # Example values; adjust as needed
    1: {"low": 3.33e-8, "high": 1.0e-2},
    2: {"low": 1.01e-2, "high": 1.0e-1},
    3: {"low": 1.01e-1, "high": 1.0},
    4: {"low": 1.001, "high": 10.0},
    5: {"low": 1.004e1, "high": 1.0e2},
    6: {"low": 1.0025e2, "high": 1.0e3},
    7: {"low": 1.002e3, "high": 7.0e7},
}

class_ranges_y2 = {
    0: {"low": 1.0e-10, "high": 1.0e-5},    # Example values; adjust as needed
    1: {"low": 1.01e-5, "high": 1.0e-4},
    2: {"low": 1.002e-4, "high": 1.0e-3},
    3: {"low": 1.002e-3, "high": 1.0e-2},
    4: {"low": 1.008e-2, "high": 1.0e-1},
    5: {"low": 1.01e-1, "high": 1.02e2},
}


# Assuming `predictions` contains the predicted classes from the model (0 to 8)
predictions_y1 = xgb_y1.predict(sequence_representations)
predictions_y2 = xgb_y2.predict(sequence_representations)
# Display the low and high values for each predicted class
for i, pred_class in enumerate(predictions_y1):
    low = class_ranges_y1[pred_class]["low"]
    high = class_ranges_y1[pred_class]["high"]
    print(f"Sample {i + 1}: Predicted Class for Kcat = {pred_class}, Range for Kcat = [{low}, {high}]")

for i, pred_class in enumerate(predictions_y2):
    low = class_ranges_y2[pred_class]["low"]
    high = class_ranges_y2[pred_class]["high"]
    print(f"Sample {i + 1}: Predicted Class for Km = {pred_class}, Range for Km = [{low}, {high}]")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:09:56] WARNING: /workspace/src/gbm/gbtree.cc:363: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:09:56] WARNING: /workspace/src/gbm/gbtree.cc:388: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:09:56] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserW

Sample 1: Predicted Class for Kcat = 0, Range for Kcat = [0.0, 3.32e-08]
Sample 1: Predicted Class for Km = 2, Range for Km = [0.0001002, 0.001]
